In [1]:
# taken from singan implementaion from repo given below
# https://github.com/FriedRonaldo/SinGAN
#
class Generator(nn.Module):
    def __init__(self, img_size_min, num_scale, size_list, bs, heads, d_model, dim1, dim2, dropout = 0.4, scale_factor=4/3):
        super(Generator, self).__init__()
        self.img_size_min = img_size_min
        self.scale_factor = scale_factor
        self.num_scale = num_scale
        self.nf = 32
        self.current_scale = 0
        self.d_model = d_model
        self.h = heads
        self.dim1 = dim1
        self.dim2 =dim2
        self.dropout = dropout
        self.nu_feat =d_model # int(d_model/self.h*2)
        self.bs = bs

        self.size_list = size_list #[int(self.img_size_min * scale_factor**i) for i in range(num_scale + 1)]
        print(self.size_list)
        self.MHA= MultiHeadAttention(bs, heads, d_model, dim1, dim2, dropout )
        self.sub_generators = nn.ModuleList()

        first_generator = nn.ModuleList()

        first_generator.append(nn.Sequential(nn.Conv2d(3, self.nf, 3, 1),
                                             nn.BatchNorm2d(self.nf),
                                             nn.LeakyReLU(2e-1)))
        for _ in range(3):
            first_generator.append(nn.Sequential(nn.Conv2d(self.nf, self.nf, 3, 1),
                                                 nn.BatchNorm2d(self.nf),
                                                 nn.LeakyReLU(2e-1)))

        first_generator.append(nn.Sequential(nn.Conv2d(self.nf, 3, 3, 1),
                                             nn.Tanh()))

        first_generator = nn.Sequential(*first_generator)

        self.sub_generators.append(first_generator)

    def forward(self, batch, img=None):
        x_list = []
        #print(batch[0].shape)
        dim2 = dim1 = batch[0].shape[-1]
        #self.MHA= MultiHeadAttention(self.bs, self.h, self.d_model,  dim1,  dim2, self.dropout )
        #x_first = self.sub_generators[0](self.MHA(batch[0]))
        #print(x_first.size())
        x_first = self.sub_generators[0](batch[0])
        x_list.append(x_first)

        if img is not None:
            x_inter = img
        else:
            x_inter = x_first

        for i in range(1, self.current_scale + 1):
            x_inter = F.interpolate(x_inter, (self.size_list[i], self.size_list[i]), mode='bilinear', align_corners=True)
            #print(x_inter.size())
            x_prev = x_inter
            x_inter = F.pad(x_inter, [5, 5, 5, 5], value=0)
            #print(self.current_scale)
            dim2 = dim1 = batch[i].shape[-1]
            """
            self.MHA= MultiHeadAttention(self.bs, self.h, self.d_model,  dim1,  dim2, self.dropout )
            x_inter = x_inter + self.MHA(batch[i])
            """
            x_inter = x_inter + (batch[i])
            x_inter = self.sub_generators[i](x_inter) + x_prev
            x_list.append(x_inter)
        #print(self.size_list)
        return x_list

    def progress(self):
        self.current_scale += 1

        if self.current_scale % 4 == 0:
            self.nf *= 2

        tmp_generator = nn.ModuleList()
        tmp_generator.append(nn.Sequential(nn.Conv2d(3, self.nf, 3, 1),
                                           nn.BatchNorm2d(self.nf),
                                           nn.LeakyReLU(2e-1)))

        for _ in range(3):
            tmp_generator.append(nn.Sequential(nn.Conv2d(self.nf, self.nf, 3, 1),
                                               nn.BatchNorm2d(self.nf),
                                               nn.LeakyReLU(2e-1)))

        tmp_generator.append(nn.Sequential(nn.Conv2d(self.nf, 3, 3, 1),
                                           nn.Tanh()))

        tmp_generator = nn.Sequential(*tmp_generator)

        if self.current_scale % 4 != 0:
            prev_generator = self.sub_generators[-1]

            # Initialize layers via copy
            if self.current_scale >= 1:
                tmp_generator.load_state_dict(prev_generator.state_dict())

        self.sub_generators.append(tmp_generator)
        print("GENERATOR PROGRESSION DONE")

NameError: name 'nn' is not defined

In [ ]:
# taken from singan implementaion from repo given below
# https://github.com/FriedRonaldo/SinGAN
#


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.nf = 32
        self.current_scale = 0

        self.sub_discriminators = nn.ModuleList()

        first_discriminator = nn.ModuleList()

        first_discriminator.append(nn.Sequential(nn.Conv2d(3, self.nf, 3, 1, 1),
                                             nn.LeakyReLU(2e-1)))
        for _ in range(3):
            first_discriminator.append(nn.Sequential(nn.Conv2d(self.nf, self.nf, 3, 1, 1),
                                                 nn.BatchNorm2d(self.nf),
                                                 nn.LeakyReLU(2e-1)))

        first_discriminator.append(nn.Sequential(nn.Conv2d(self.nf, 1, 3, 1, 1)))

        first_discriminator = nn.Sequential(*first_discriminator)

        self.sub_discriminators.append(first_discriminator)

    def forward(self, x):
        out = self.sub_discriminators[self.current_scale](x)
        return out

    def progress(self):
        self.current_scale += 1
        # Lower scale discriminators are not used in later ... replace append to assign?
        if self.current_scale % 4 == 0:
            self.nf *= 2

        tmp_discriminator = nn.ModuleList()
        tmp_discriminator.append(nn.Sequential(nn.Conv2d(3, self.nf, 3, 1, 1),
                                               nn.LeakyReLU(2e-1)))

        for _ in range(3):
            tmp_discriminator.append(nn.Sequential(nn.Conv2d(self.nf, self.nf, 3, 1, 1),
                                                   nn.BatchNorm2d(self.nf),
                                                   nn.LeakyReLU(2e-1)))

        tmp_discriminator.append(nn.Sequential(nn.Conv2d(self.nf, 1, 3, 1, 1)))

        tmp_discriminator = nn.Sequential(*tmp_discriminator)

        if self.current_scale % 4 != 0:
            prev_discriminator = self.sub_discriminators[-1]

            # Initialize layers via copy
            if self.current_scale >= 1:
                tmp_discriminator.load_state_dict(prev_discriminator.state_dict())

        self.sub_discriminators.append(tmp_discriminator)
        print("DISCRIMINATOR PROGRESSION DONE")

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def compute_grad_gp(d_out, x_in):
    batch_size = x_in.size(0)
    grad_dout = autograd.grad(
        outputs=d_out.sum(), inputs=x_in,grad_outputs=torch.ones_like(d_out.sum()),
        create_graph=True, retain_graph=True, only_inputs=True)[0]
    grad_dout2 = grad_dout.pow(2)
    assert(grad_dout2.size() == x_in.size())
    reg = grad_dout2.view(batch_size, -1).sum(1)
    return reg


def compute_grad_gp_wgan(D, x_real, x_fake, gpu):
    alpha = torch.rand(x_real.size(0), 1, 1, 1).cuda(gpu)

    x_interpolate = ((1 - alpha) * x_real + alpha * x_fake).detach()
    x_interpolate.requires_grad = True
    d_inter_logit = D(x_interpolate)
    grad = torch.autograd.grad(d_inter_logit, x_interpolate,
                               grad_outputs=torch.ones_like(d_inter_logit), create_graph=True)[0]

    norm = grad.view(grad.size(0), -1).norm(p=2, dim=1)

    d_gp = ((norm - 1) ** 2).mean()
    return d_gp

In [ ]:
# taken from singan implementaion from repo given below
# https://github.com/FriedRonaldo/SinGAN
#

from tqdm import trange
import torchvision.utils as vutils


def trainSinGAN(data_loader, networks, opts, stage, args, additional):
  # avg meter
  d_losses = AverageMeter()
  g_losses = AverageMeter()

  # set nets
  D = networks[0]
  G = networks[1]
  # set opts
  d_opt = opts['d_opt']
  g_opt = opts['g_opt']
  # switch to train mode
  D.train()
  G.train()
  # summary writer
  # writer = additional[0]
  train_it = iter(data_loader)
  # total_iter = 2000 * (args.num_scale - stage + 1)
  # decay_lr = 1600 * (args.num_scale - stage + 1)
  total_iter = 2000
  decay_lr = 4

  d_iter = 3
  g_iter = 3

  t_train = trange(0, total_iter, initial=0, total=total_iter)
  x_in = next(train_it)

  x_in = x_in.cuda( non_blocking=True)
  x_org = x_in
  x_in = F.interpolate(x_in, (args.size_list[stage], args.size_list[stage]), mode='bilinear', align_corners=True)
  #x_0 = F.interpolate(x_0, (args.size_list[stage], args.size_list[stage]), mode='bilinear', align_corners=True)
  
  

  """
  z_rec = additional['z_rec']

  for z_idx in range(len(z_rec)):
      z_rec[z_idx] = z_rec[z_idx].cuda(args.gpu, non_blocking=True)

  x_in = next(train_it)

  x_in = x_in.cuda(args.gpu, non_blocking=True)
  x_org = x_in
  x_in = F.interpolate(x_in, (args.size_list[stage], args.size_list[stage]), mode='bilinear', align_corners=True)
  vutils.save_image(x_in.detach().cpu(), os.path.join(args.res_dir, 'ORGTRAIN_{}.png'.format(stage)),
                    nrow=1, normalize=True)
   """
  x_in_list = []

  for xidx in range(0, stage + 1):
      x_tmp = F.interpolate(x_org, (args.size_list[xidx], args.size_list[xidx]), mode='bilinear', align_corners=True)
      x_tmp = F.pad(x_tmp, [5, 5, 5, 5], value=0)
      x_in_list.append(x_tmp)
  
  x_in_list_nopad = []
  
  for xidx in range(0, stage + 1):
      x_tmp = F.interpolate(x_org, (args.size_list[xidx], args.size_list[xidx]), mode='bilinear', align_corners=True)
      
      x_in_list_nopad.append(x_tmp)
  z_rec = F.pad(x_in, [5, 5, 5, 5], value=0)
  for i in t_train:
      if i == decay_lr:
          for param_group in d_opt.param_groups:
                  param_group['lr'] *= 0.1
                  print("DISCRIMINATOR LEARNING RATE UPDATE TO :", param_group['lr'])
          for param_group in g_opt.param_groups:
                  param_group['lr'] *= 0.1
                  print("GENERATOR LEARNING RATE UPDATE TO :", param_group['lr'])

      for _ in range(g_iter):
          g_opt.zero_grad()
          #print(type(z_rec))
          x_rec_list = G(x_in_list)
          #print(x_rec_list[-1].shape)
          #print(x_in.shape)
          g_rec = F.mse_loss(x_rec_list[-1], x_in)
          # calculate rmse for each scale
          rmse_list = [1.0]
          for rmseidx in range(1, stage + 1):
              rmse = torch.sqrt(F.mse_loss(x_rec_list[rmseidx], x_in_list_nopad[rmseidx]))
              rmse_list.append(rmse)

          z_list = [F.pad(rmse_list[z_idx] * torch.randn(args.batch_size, 3, args.size_list[z_idx],
                                              args.size_list[z_idx]).cuda(args.gpu, non_blocking=True),
                          [5, 5, 5, 5], value=0) for z_idx in range(stage + 1)]
          
          x_fake_list = G(z_list )
          #print(len(x_fake_list[-1]))

          g_fake_logit = D(x_fake_list[-1])

          ones = torch.ones_like(g_fake_logit).cuda(args.gpu)

          if args.gantype == 'wgangp':
              # wgan gp
              g_fake = -torch.mean(g_fake_logit, (2, 3))
              g_loss = g_fake + 10.0 * g_rec
          elif args.gantype == 'zerogp':
              # zero centered GP
              g_fake = F.binary_cross_entropy_with_logits(g_fake_logit, ones, reduction='none').mean()
              g_loss = g_fake + 100.0 * g_rec

          elif args.gantype == 'lsgan':
              # lsgan
              g_fake = F.mse_loss(torch.mean(g_fake_logit, (2, 3)), 0.9 * ones)
              g_loss = g_fake + 50.0 * g_rec

          g_loss.backward(retain_graph = True)
          g_opt.step()

          g_losses.update(g_loss.item(), x_in.size(0))

      # Update discriminator
      for _ in range(d_iter):
          x_in.requires_grad = True

          d_opt.zero_grad()
          x_fake_list =  G(z_list )
          

          d_fake_logit = g_fake_logit
          d_real_logit = D(x_in)

          ones = torch.ones_like(d_real_logit).cuda(args.gpu)
          zeros = torch.zeros_like(d_fake_logit).cuda(args.gpu)

          if args.gantype == 'wgangp':
              # wgan gp
              d_fake = torch.mean(d_fake_logit, (2, 3))
              d_real = -torch.mean(d_real_logit, (2, 3))
              d_gp = compute_grad_gp_wgan(D, x_in, x_fake_list[-1], args.gpu)
              d_loss = d_real + d_fake + 0.1 * d_gp
          elif args.gantype == 'zerogp':
              # zero centered GP
              # d_fake = F.binary_cross_entropy_with_logits(torch.mean(d_fake_logit, (2, 3)), zeros)
              d_fake = F.binary_cross_entropy_with_logits(d_fake_logit, zeros, reduction='none').mean()
              # d_real = F.binary_cross_entropy_with_logits(torch.mean(d_real_logit, (2, 3)), ones)
              d_real = F.binary_cross_entropy_with_logits(d_real_logit, ones, reduction='none').mean()
              #d_gp = compute_grad_gp(torch.mean(d_real_logit, (2, 3)), x_in)
              d_loss = d_real + d_fake# + 10.0 * d_gp

          elif args.gantype == 'lsgan':
              # lsgan
              d_fake = F.mse_loss(torch.mean(d_fake_logit, (2, 3)), zeros)
              d_real = F.mse_loss(torch.mean(d_real_logit, (2, 3)), 0.9 * ones)
              d_loss = d_real + d_fake

          d_loss.backward(retain_graph=True)
          d_opt.step()

          d_losses.update(d_loss.item(), x_in.size(0))

      t_train.set_description('Stage: [{}/{}] Avg Loss: D[{d_losses.avg:.3f}] G[{g_losses.avg:.3f}] RMSE[{rmse:.3f}]'
                              .format(stage, args.num_scale, d_losses=d_losses, g_losses=g_losses, rmse=rmse_list[-1]))

In [ ]:
with open('/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat '/gdrive/My Drive/foo.txt'

In [ ]:
class Arguments():
  def __init__(self,gpu = None,gantype = 'zerogp',model_name = 'SinGan', batch_size = 128, img_size_max= 224, img_size_min=20,
               load_model = None, multiprocessing_distributed= None, world_size=1, log_dir= None, res_dir = None, num_scale = None):
    self.gpu = gpu
    self.gantype = gantype
    self.model_name = "SinGan"
    self.batch_size = batch_size
    self.img_size_max= img_size_max
    self.img_size_min=img_size_min
    self.load_model = load_model
    self.multiprocessing_distributed= multiprocessing_distributed 
    self.world_size=world_size
    self.log_dir= log_dir
    self.res_dir = res_dir 
    self.num_scale = num_scale 
    self.stage = None
    self.distributed = None
    self.workers = 8
    self.d_model = None
    self.h = None
    self.dim1 = None
    self.dim2 = None
    self.dropout = None
    self.nu_feat =None
    self.dataset = None
    



In [ ]:

def save_checkpoint(state, check_list, log_dir, epoch=0):
    check_file = os.path.join(log_dir, 'model_{}.ckpt'.format(epoch))
    torch.save(state, check_file)
    check_list.write('model_{}.ckpt\n'.format(epoch))

def makedirs(path):
    if not os.path.exists(path):
        os.makedirs(path)
def formatted_print(notice, value):
    print('{0:<40}{1:<40}'.format(notice, value))



In [ ]:

def main_worker(gpu, ngpus_per_node, args):
    args.gpu = 0


    if args.gpu is not None:
        print("Use GPU: {} for training".format(args.gpu))
    args.distributed = False
    if args.distributed:
        if args.multiprocessing_distributed:
            args.rank = args.rank * ngpus_per_node + gpu
        dist.init_process_group(backend='nccl', init_method='tcp://127.0.0.1:'+args.port,
                                world_size=args.world_size, rank=args.rank)

    ################
    # Define model #
    ################
    # 4/3 : scale factor in the paper
    scale_factor = 4/3



    tmp_scale = args.img_size_max / args.img_size_min
    args.num_scale = int(np.round(np.log(tmp_scale) / np.log(scale_factor)))
    args.size_list = [int(args.img_size_min * scale_factor**i) for i in range(args.num_scale + 1)]
    #args.size_list = [int(args.img_size_min *i) for i in range(1,args.num_scale + 1)]

    discriminator = Discriminator()
    generator = Generator(args.img_size_min, args.num_scale, args.size_list, args.batch_size, args.heads, args.d_model, args.dim1, args.dim2, args.dropout,  scale_factor)

    networks = [discriminator, generator]

    if args.distributed:
        if args.gpu is not None:
            print('Distributed to', args.gpu)
            torch.cuda.set_device(args.gpu)
            networks = [x.cuda(args.gpu) for x in networks]
            args.batch_size = int(args.batch_size / ngpus_per_node)
            args.workers = int(args.workers / ngpus_per_node)
            networks = [torch.nn.parallel.DistributedDataParallel(x, device_ids=[args.gpu], output_device=args.gpu) for x in networks]
        else:
            networks = [x.cuda() for x in networks]
            networks = [torch.nn.parallel.DistributedDataParallel(x) for x in networks]

    elif args.gpu is not None:
        torch.cuda.set_device(args.gpu)
        networks = [x.cuda(args.gpu) for x in networks]
    else:
        networks = [torch.nn.DataParallel(x).cuda() for x in networks]

    discriminator, generator, = networks

    ######################
    # Loss and Optimizer #
    ######################
    if args.distributed:
        d_opt = torch.optim.Adam(discriminator.module.sub_discriminators[0].parameters(), 5e-4, (0.5, 0.999))
        g_opt = torch.optim.Adam(generator.module.sub_generators[0].parameters(), 5e-4, (0.5, 0.999))
    else:
        d_opt = torch.optim.Adam(discriminator.sub_discriminators[0].parameters(), 5e-4, (0.5, 0.999))
        g_opt = torch.optim.Adam(generator.sub_generators[0].parameters(), 5e-4, (0.5, 0.999))

    ##############
    # Load model #
    ##############
    args.stage = 0
    if args.load_model is not None:
        check_load = open(os.path.join(args.log_dir, "checkpoint.txt"), 'r')
        to_restore = check_load.readlines()[-1].strip()
        load_file = os.path.join(args.log_dir, to_restore)
        if os.path.isfile(load_file):
            print("=> loading checkpoint '{}'".format(load_file))
            checkpoint = torch.load(load_file, map_location='cpu')
            for _ in range(int(checkpoint['stage'])):
                generator.progress()
                discriminator.progress()
            networks = [discriminator, generator]
            if args.distributed:
                if args.gpu is not None:
                    print('Distributed to', args.gpu)
                    torch.cuda.set_device(args.gpu)
                    networks = [x.cuda(args.gpu) for x in networks]
                    args.batch_size = int(args.batch_size / ngpus_per_node)
                    args.workers = int(args.workers / ngpus_per_node)
                    networks = [
                        torch.nn.parallel.DistributedDataParallel(x, device_ids=[args.gpu], output_device=args.gpu) for
                        x in networks]
                else:
                    networks = [x.cuda() for x in networks]
                    networks = [torch.nn.parallel.DistributedDataParallel(x) for x in networks]

            elif args.gpu is not None:
                torch.cuda.set_device(args.gpu)
                networks = [x.cuda(args.gpu) for x in networks]
            else:
                networks = [torch.nn.DataParallel(x).cuda() for x in networks]

            discriminator, generator, = networks

            args.stage = checkpoint['stage']
            args.img_to_use = checkpoint['img_to_use']
            discriminator.load_state_dict(checkpoint['D_state_dict'])
            generator.load_state_dict(checkpoint['G_state_dict'])
            d_opt.load_state_dict(checkpoint['d_optimizer'])
            g_opt.load_state_dict(checkpoint['g_optimizer'])
            print("=> loaded checkpoint '{}' (stage {})"
                  .format(load_file, checkpoint['stage']))
        else:
            print("=> no checkpoint found at '{}'".format(args.log_dir))

    cudnn.benchmark = True

    ###########
    # Dataset #
    ###########
    transformed_dataset = CityscapeDataset(
                                           transforms=composed_transforms
                                           )

    if args.distributed:
        train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
    else:
        train_sampler = None

    train_loader = DataLoader(transformed_dataset, batch_size = args.batch_size,
                            sampler=torch.utils.data.SequentialSampler(transformed_dataset) ,  num_workers=args.workers)
  

    ######################
    # Validate and Train #
    ######################
    z_fix_list = [F.pad(torch.randn(args.batch_size, 3, args.size_list[0], args.size_list[0]), [5, 5, 5, 5], value=0)]
    zero_list = [F.pad(torch.zeros(args.batch_size, 3, args.size_list[zeros_idx], args.size_list[zeros_idx]),
                       [5, 5, 5, 5], value=0) for zeros_idx in range(1, args.num_scale + 1)]
    z_fix_list = z_fix_list + zero_list
    """
    if args.validation:
        validateSinGAN(train_loader, networks, args.stage, args, {"z_rec": z_fix_list})
        return

    elif args.test:
        validateSinGAN(train_loader, networks, args.stage, args, {"z_rec": z_fix_list})
        return
    """

    if not args.multiprocessing_distributed or (args.multiprocessing_distributed and args.rank % ngpus_per_node == 0):
        check_list = open(os.path.join(args.log_dir, "checkpoint.txt"), "a+")
        record_txt = open(os.path.join(args.log_dir, "record.txt"), "a+")
        record_txt.write('DATASET\t:\t{}\n'.format(args.dataset))
        record_txt.write('GANTYPE\t:\t{}\n'.format(args.gantype))
#        record_txt.write('IMGTOUSE\t:\t{}\n'.format(args.img_to_use))
        record_txt.close()

    for stage in range(args.stage, args.num_scale + 1):
        if args.distributed:
            train_sampler.set_epoch(stage)

        trainSinGAN(train_loader, networks, {"d_opt": d_opt, "g_opt": g_opt}, stage, args, {"z_rec": z_fix_list})
       # validateSinGAN(train_loader, networks, stage, args, {"z_rec": z_fix_list})

        if args.distributed:
            discriminator.module.progress()
            generator.module.progress()
        else:
            discriminator.progress()
            generator.progress()

        networks = [discriminator, generator]

        if args.distributed:
            if args.gpu is not None:
                print('Distributed', args.gpu)
                torch.cuda.set_device(args.gpu)
                networks = [x.cuda(args.gpu) for x in networks]
                args.batch_size = int(args.batch_size / ngpus_per_node)
                args.workers = int(args.workers / ngpus_per_node)
                networks = [torch.nn.parallel.DistributedDataParallel(x, device_ids=[args.gpu], output_device=args.gpu)
                            for x in networks]
            else:
                networks = [x.cuda() for x in networks]
                networks = [torch.nn.parallel.DistributedDataParallel(x) for x in networks]

        elif args.gpu is not None:
            torch.cuda.set_device(args.gpu)
            networks = [x.cuda(args.gpu) for x in networks]
        else:
            networks = [torch.nn.DataParallel(x).cuda() for x in networks]

        discriminator, generator, = networks

        # Update the networks at finest scale
        if args.distributed:
            for net_idx in range(generator.module.current_scale):
                for param in generator.module.sub_generators[net_idx].parameters():
                    param.requires_grad = False
                for param in discriminator.module.sub_discriminators[net_idx].parameters():
                    param.requires_grad = False

            d_opt = torch.optim.Adam(discriminator.module.sub_discriminators[discriminator.current_scale].parameters(),
                                     5e-4, (0.5, 0.999))
            g_opt = torch.optim.Adam(generator.module.sub_generators[generator.current_scale].parameters(),
                                     5e-4, (0.5, 0.999))
        else:
            for net_idx in range(generator.current_scale):
                for param in generator.sub_generators[net_idx].parameters():
                    param.requires_grad = False
                for param in discriminator.sub_discriminators[net_idx].parameters():
                    param.requires_grad = False

            d_opt = torch.optim.Adam(discriminator.sub_discriminators[discriminator.current_scale].parameters(),
                                     5e-4, (0.5, 0.999))
            g_opt = torch.optim.Adam(generator.sub_generators[generator.current_scale].parameters(),
                                     5e-4, (0.5, 0.999))

        ##############
        # Save model #
        ##############
        if not args.multiprocessing_distributed or (args.multiprocessing_distributed and args.rank % ngpus_per_node == 0):
            if stage == 0:
                check_list = open(os.path.join(args.log_dir, "checkpoint.txt"), "a+")
            save_checkpoint({
                'stage': stage + 1,
                'D_state_dict': discriminator.state_dict(),
                'G_state_dict': generator.state_dict(),
                'd_optimizer': d_opt.state_dict(),
                'g_optimizer': g_opt.state_dict()
                
            }, check_list, args.log_dir, stage + 1)
            if stage == args.num_scale:
                check_list.close()

    return generator


In [ ]:
args = Arguments()
args.gpu=0
args.distributed = False
args.load_model = None
args.batch_size = 2
args.img_size_max =  224
args.img_size_min = 28
args.heads = 8
args.d_model = 3
args.nu_feat =3
args.dim1 = 224
args.dim2 = 224
args.dropout = 0.4 
args.dataset = "train"

if args.gpu is not None:
   
    warnings.warn('You have chosen a specific GPU. This will completely '
                  'disable data parallelism.')
"""
args.distributed = args.world_size > 1 or args.multiprocessing_distributed


"""
ngpus_per_node = torch.cuda.device_count()
args.load_model = None
if args.load_model is None:
    args.model_name = '{}_{}'.format(args.model_name, datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
else:
    args.model_name = args.load_model

makedirs(home/'logs')
makedirs(home/'results')

args.log_dir = os.path.join(home,'./logs', args.model_name)
args.res_dir = os.path.join(home,'./results', args.model_name)

makedirs(args.log_dir)
makedirs(os.path.join(args.log_dir, 'codes'))
makedirs(os.path.join(args.log_dir, 'codes', 'models'))
makedirs(args.res_dir)

if args.load_model is None:
    pyfiles = glob("./*.py")
    modelfiles = glob('./models/*.py')
    for py in pyfiles:
        copyfile(py, os.path.join(args.log_dir, 'codes') + "/" + py)
    for py in modelfiles:
        copyfile(py, os.path.join(args.log_dir, 'codes', py[2:]))

formatted_print('Total Number of GPUs:', ngpus_per_node)
formatted_print('Total Number of Workers:', args.workers)
formatted_print('Batch Size:', args.batch_size)
formatted_print('Max image Size:', args.img_size_max)
formatted_print('Min image Size:', args.img_size_min)
formatted_print('Log DIR:', args.log_dir)
formatted_print('Result DIR:', args.res_dir)
formatted_print('GAN TYPE:', args.gantype)

if args.multiprocessing_distributed:
    args.world_size = ngpus_per_node * args.world_size
    mp.spawn(main_worker, nprocs=ngpus_per_node, args=(ngpus_per_node, args))
else:
    generator = main_worker(args.gpu, ngpus_per_node, args)



In [ ]:
def evaluate(stage, generator):
  train_it = iter(data_loader)

  x_in = next(train_it)

  x_in = x_in.cuda( non_blocking=True)
  x_org = x_in
  
  x_in = F.interpolate(x_in, (args.size_list[stage], args.size_list[stage]), mode='bilinear', align_corners=True)

  x_in_list = []
  print(len(args.size_list))
  for xidx in range(0, stage + 1):
      x_tmp = F.interpolate(x_org, (args.size_list[xidx], args.size_list[xidx]), mode='bilinear', align_corners=True)
      x_tmp = F.pad(x_tmp, [5, 5, 5, 5], value=0)
      x_in_list.append(x_tmp)
  generator.eval()
  out = generator(x_in_list)
  return out
  